In [2]:
import numpy as np
import pandas as pd
import random
from datetime import datetime
import matplotlib.pyplot as plt
from pprint import pprint

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

import warnings
warnings.filterwarnings("ignore")

2023-05-06 15:07:42.190948: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-06 15:07:42.217566: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-06 15:07:42.219128: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-06 15:07:42.902321: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
training_data = pd.read_csv("./DateDatasets/training.csv").drop(columns=["Unnamed: 0"])
testing_data = pd.read_csv("./DateDatasets/testing.csv").drop(columns=["Unnamed: 0"])
validating_data = pd.read_csv("./DateDatasets/validation.csv").drop(columns=["Unnamed: 0"])

In [1]:
cols = ["StationEnd", "Count", "Count1", "Count1week", "Temp", "ATemp", "Hour", "Windspeed"]

training_data_met9 = training_data[[col for col in training_data.columns if col in cols]]
testing_data_met9 = testing_data[[col for col in testing_data.columns if col in cols]]

trainX = training_data_met9.drop(columns=["Count"])
trainY = training_data_met9["Count"]

testX = testing_data_met9.drop(columns=["Count"])
testY = testing_data_met9["Count"]

trainX = np.reshape(trainX.values, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX.values, (testX.shape[0], 1, testX.shape[1]))

model = Sequential()
model.add(LSTM(4, input_shape=(1,testing_data_met9.shape[1]-1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=40, batch_size=1, verbose=2)

scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(pd.concat([training_data_met9, testing_data_met9]))

# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

NameError: name 'training_data' is not defined

In [ ]:
trainPredict = model.predict(trainX)

trainScore = np.sqrt(mean_squared_error(trainY[:], trainPredict[:]))
print('Train Score: %.2f RMSE' % (trainScore))
results = model.evaluate(trainX, trainY, batch_size=128)
print("Train Accuracy:", results)

NameError: name 'model' is not defined

In [ ]:
testPredict = model.predict(testX)

testScore = np.sqrt(mean_squared_error(testY[:], testPredict[:]))
print('Test Score: %.2f RMSE' % (testScore))
results = model.evaluate(testX, testY, batch_size=128)
print("Test Accuracy:", results)

NameError: name 'model' is not defined

In [ ]:
valX = validating_data.drop(columns=["Count"])
valY = validating_data["Count"]

valPredict = model.predict(valX)

valScore = np.sqrt(mean_squared_error(valY[:], valPredict[:]))
print('Validation Score: %.2f RMSE' % (valScore))
results = model.evaluate(valX, valY, batch_size=128)
print("Validation Accuracy:", results)

NameError: name 'validating_data' is not defined